In [1]:
%env CUDA_VISIBLE_DEVICES=''

import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from IPython.core.display import display, HTML

env: CUDA_VISIBLE_DEVICES=''


## Load the trained model

In [2]:
checkout_dir = 'checkpoints0/model25000'

text, words_to_ids, ids_to_words = pickle.load(open('data/preprocess.p', mode='rb'))
sess = tf.Session()

loader = tf.train.import_meta_graph(checkout_dir + '.meta')
loader.restore(sess, checkout_dir)

loaded_graph = tf.get_default_graph()
# Get Tensors from loaded model
names = ["input:0", "initial_state:0", "final_state:0", "probs:0"]
tensors = [loaded_graph.get_tensor_by_name(name) for name in names]
inputs, initial_state, final_state, probs = tensors

INFO:tensorflow:Restoring parameters from checkpoints0/model25000


In [3]:
from tokenizer import *

def pick_word(probabilities, int_to_vocab):
    probabilities[words_to_ids['not_in_vocab']] = 0.
    # make it sum to 1
    probabilities /= probabilities.sum()
    word_id = np.random.choice(np.arange(len(probabilities)), size=1, p=probabilities)[0]
    #word_id = np.argmax(probabilities)
    return int_to_vocab[word_id]

def generate(starting_text='Homer Simpson:', generate_length=300):
    sentence_tokens = text_to_tokens(starting_text)
    prev_state = sess.run(initial_state, {inputs: np.array([[1]])})

    for n in range(generate_length):
        sentence_ids = [[words_to_ids[word] for word in sentence_tokens]]
        sentence_len = len(sentence_ids[0])

        probabilities, prev_state = sess.run(
            [probs, final_state],
            {inputs: sentence_ids, initial_state: prev_state})

        predicted_token = pick_word(probabilities[0][sentence_len-1], ids_to_words)
        sentence_tokens.append(predicted_token)
    return tokens_to_text(sentence_tokens)

## Generate an original script from a trained network

In [4]:
script = generate(starting_text='Homer Simpson:', generate_length=500)

display(HTML('<pre>%s</pre>' % script))